In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle

In [2]:
data=pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
gen_encoder=LabelEncoder()
data['Gender']=gen_encoder.fit_transform(data['Gender'])

In [5]:
geo_encoder=OneHotEncoder(handle_unknown='ignore')
geo_encoded=geo_encoder.fit_transform(data[['Geography']]).toarray()
geo_encoded_df=pd.DataFrame(geo_encoded,columns=geo_encoder.get_feature_names_out(['Geography']))


In [6]:
geo_encoded_df


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [7]:
data=pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [8]:
X=data.drop('EstimatedSalary',axis=1)
y=data['EstimatedSalary']


In [9]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [10]:
with open('label_encoder_gender_reg.pkl','wb') as f:
    pickle.dump(gen_encoder,f)

with open('onehotencoder_geo_reg.pkl','wb') as f:
    pickle.dump(geo_encoder,f)

with open('scaler_reg.pkl','wb') as f:
    pickle.dump(scaler,f)

In [11]:
# Regression ANN model

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


In [12]:
# Define the model

model=Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)),
    Dense(32,activation='relu'),
    Dense(1) # Linear activation function applied by default, output layer
])

model.compile(optimizer='adam',loss='mean_squared_error',metrics=['mae'])
# MAE is used as the loss function as it is a regression problem

model.summary()


d:\Proj1\venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [13]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

# Set up Tensorboard for logs
log_dir="reglogs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=tf.keras.callbacks.TensorBoard(log_dir=log_dir,histogram_freq=1)


In [14]:
# Set up Early Stopping
# This will stop the training if the model is not learning anything new after many epochs
early_stopping=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True) # monitoring validation loss

In [15]:
# Train the model

history=model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=100,callbacks=[early_stopping,tensorflow_callback])

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 13546117120.0000 - mae: 100957.9844 - val_loss: 13025485824.0000 - val_mae: 98611.5859
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 13305867264.0000 - mae: 99977.7891 - val_loss: 12837520384.0000 - val_mae: 97659.8438
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 13072460800.0000 - mae: 98735.7188 - val_loss: 12333536256.0000 - val_mae: 95111.2031
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 12501928960.0000 - mae: 96139.0625 - val_loss: 11447201792.0000 - val_mae: 90619.0938
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 11576297472.0000 - mae: 91289.5391 - val_loss: 10210577408.0000 - val_mae: 84400.4141
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 9996909568.0000 - mae: 83693.6719 - val_loss: 8742421504.0000 - val_mae: 76988.0781
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 8527484416.0000 - mae: 76393.1875 - val_loss: 7233434112.0

In [16]:
%load_ext tensorboard
%tensorboard --logdir reglogs/fit

Reusing TensorBoard on port 6006 (pid 47976), started 0:10:00 ago. (Use '!kill 47976' to kill it.)

In [17]:
# Evaluate the model

test_loss,test_mae=model.evaluate(X_test,y_test)
print(f'Test MAE: {test_mae}')

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 810us/step - loss: 3416595968.0000 - mae: 50806.2656
Test MAE: 50104.49609375


In [18]:
model.save('regression_model.h5')